In [ ]:
## Creates .csv list of crypto tickers        ## As of 2021/04/30 There are: 4904 cryptos ##

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm.notebook import tqdm

class gatherers:
    
    def get_cryptos(self, path):
        driver = webdriver.Chrome(path)
        wait = WebDriverWait(driver, 10)
        
        driver.get("https://ca.investing.com/crypto/currencies")
        
        try:    
            popup_close = wait.until(EC.element_to_be_clickable((By.XPATH, "//div/div/i[contains(@class, 'popupCloseIcon')]")))
            popup_close.click()
        except:
            pass
            ## Wait for all rows to load
        num_rows = driver.find_element_by_xpath("//*[@id='fullColumn']/ul/li[1]/span[2]/a").text
        num_rows = int(num_rows.replace(',', ''))
        
        wait.until(EC.visibility_of_element_located((By.XPATH, "//table/tbody/tr[" + str(num_rows-1) + "]")))
        rows = driver.find_elements_by_xpath("//table/tbody/tr")
        driver.close
        
            ## Begin Scraping ##
        
        with open("./../Tickers/tickers_crypto.csv", "w") as f:
            for i in tqdm(range(len(rows)), desc = 'Scraping Rows'):
                
                try:
                    symbol = rows[i].find_element_by_xpath(".//td[contains(@class, 'js-currency-symbol')]").get_attribute("title")
                    f.write(symbol + "\n")
                except:
                    pass
    
if __name__ == "__main__": 
    mac_path = "/opt/homebrew/bin/chromedriver"
    win_path = "C:/WebDriver/chromedriver"
    
    gatherers.get_cryptos(mac_path)
